In [145]:
# !pip install TensorFlow 


# Задание 10

In [146]:
# !pip install -q numpy tensorflow_datasets tensorflow

In [147]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow import keras

In [148]:
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics

Импортируем данную библиотеку, а затем с помощью нее загрузим набор данных "titanic". В процессе загрузки сразу разделим его на тренировочную, валидационную и тестовую выборку.


In [149]:
import tensorflow_datasets as tfds
ds_train_tf, ds_validation_tf, ds_test_tf, ds_prediction_tf = tfds.load(
    name='titanic',
    split=['train[:70%]', 'train[70%:80%]', 'train[80%:90%]', 'train[90%:]']
)

Выведем первую строку тренировочного набора данных для ознакомления.

In [150]:
for x, y in train_ds.take(1):
    print(f"x={x}")
    print(f"y={y}")

x={'age': <tf.Tensor: shape=(), dtype=float32, numpy=30.0>, 'boat': <tf.Tensor: shape=(), dtype=string, numpy=b'Unknown'>, 'body': <tf.Tensor: shape=(), dtype=int32, numpy=-1>, 'cabin': <tf.Tensor: shape=(), dtype=string, numpy=b'Unknown'>, 'embarked': <tf.Tensor: shape=(), dtype=int64, numpy=2>, 'fare': <tf.Tensor: shape=(), dtype=float32, numpy=13.0>, 'home.dest': <tf.Tensor: shape=(), dtype=string, numpy=b'Sarnia, ON'>, 'name': <tf.Tensor: shape=(), dtype=string, numpy=b'McCrie, Mr. James Matthew'>, 'parch': <tf.Tensor: shape=(), dtype=int32, numpy=0>, 'pclass': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'sex': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'sibsp': <tf.Tensor: shape=(), dtype=int32, numpy=0>, 'ticket': <tf.Tensor: shape=(), dtype=string, numpy=b'233478'>}
y=0


## Предобработка данных

In [151]:
ds_train = tfds.as_dataframe(ds_train_tf)
ds_val = tfds.as_dataframe(ds_validation_tf)
ds_test = tfds.as_dataframe(ds_test_tf)

In [152]:
y_train = ds_train['survived']
y_val = ds_val['survived']
y_test = ds_test['survived']

In [153]:
ds_train = ds_train.drop(['survived'], axis=1)
ds_test = ds_test.drop(['survived'], axis=1)
ds_val = ds_val.drop(['survived'], axis=1)

In [154]:
ds_train

,age,boat,body,cabin,embarked,fare,home.dest,name,parch,pclass,sex,sibsp,ticket
0,30.0,b'Unknown',-1,b'Unknown',2,13.000000,"b'Sarnia, ON'","b'McCrie, Mr. James Matthew'",0,1,0,0,b'233478'
1,37.0,b'Unknown',98,b'Unknown',2,7.925000,"b'Ruotsinphytaa, Finland New York, NY'","b'Gustafsson, Mr. Anders Vilhelm'",0,2,0,2,b'3101276'
2,28.0,b'9',-1,b'Unknown',2,13.000000,b'Spain',"b'Reynaldo, Ms. Encarnacion'",0,1,1,0,b'230434'
3,18.0,b'Unknown',-1,b'Unknown',2,73.500000,"b'Lyndhurst, England'","b'Davies, Mr. Charles Henry'",0,1,0,0,b'S.O.C. 14879'
4,-1.0,b'Unknown',-1,b'Unknown',0,7.895800,b'Unknown',"b'Gheorgheff, Mr. Stanio'",0,2,0,0,b'349254'
...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,21.0,b'12',-1,b'Unknown',2,10.500000,"b'Guernsey / Wilmington, DE'","b'Rugg, Miss. Emily'",0,1,1,0,b'C.A. 31026'
912,36.0,b'Unknown',-1,b'Unknown',2,7.895800,b'Unknown',"b'Turcin, Mr. Stjepan'",0,2,0,0,b'349247'
913,-1.0,b'Unknown',-1,b'Unknown',2,7.550000,b'Unknown',"b'MacKay, Mr. George William'",0,2,0,0,b'C.A. 42795'
914,39.0,b'2',-1,b'Unknown',2,211.337494,b'Unknown',"b'Kreuchen, Miss. Emilie'",0,0,1,0,b'24160'


In [155]:
print(f"Lengths: {len(ds_train)}, {len(ds_test)}")

Lengths: 916, 131


In [156]:
ds_train.describe()

,age,body,embarked,fare,parch,pclass,sex,sibsp
count,916.000000,916.000000,916.000000,916.000000,916.000000,916.000000,916.000000,916.000000
mean,23.807041,15.080786,1.485808,34.101959,0.395197,1.292576,0.346070,0.496725
std,18.128685,58.199976,0.822375,54.664959,0.878983,0.841278,0.475976,1.059431
min,-1.000000,-1.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000
25%,5.000000,-1.000000,1.000000,7.895800,0.000000,0.750000,0.000000,0.000000
50%,24.750000,-1.000000,2.000000,14.454200,0.000000,2.000000,0.000000,0.000000
75%,36.000000,-1.000000,2.000000,31.303125,0.000000,2.000000,1.000000,1.000000
max,80.000000,327.000000,3.000000,512.329224,9.000000,2.000000,1.000000,8.000000


In [157]:
ds_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 916 entries, 0 to 915
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        916 non-null    float32
 1   boat       916 non-null    object 
 2   body       916 non-null    int32  
 3   cabin      916 non-null    object 
 4   embarked   916 non-null    int64  
 5   fare       916 non-null    float32
 6   home.dest  916 non-null    object 
 7   name       916 non-null    object 
 8   parch      916 non-null    int32  
 9   pclass     916 non-null    int64  
 10  sex        916 non-null    int64  
 11  sibsp      916 non-null    int32  
 12  ticket     916 non-null    object 
dtypes: float32(2), int32(3), int64(3), object(5)
memory usage: 75.3+ KB


In [158]:
num_features = ['age', 'fare']
# Не берем только body и name
cat_features = ['boat', 'cabin', 'embarked', 'home.dest', 'parch', 'pclass', 'sex',	'sibsp', 'ticket']

Для работы с категориальными признаками воспользуемся библиотекой Target Encoder.

In [159]:
!pip install category_encoders -q

In [160]:
import category_encoders as ce

In [161]:
te = ce.target_encoder.TargetEncoder(cols=cat_features)
ds_train = te.fit_transform(ds_train, y_train)
ds_val = te.transform(ds_val)
ds_test = te.transform(ds_test)

In [162]:
ds_train

,age,boat,body,cabin,embarked,fare,home.dest,name,parch,pclass,sex,sibsp,ticket
0,30.0,0.022147,-1,0.288326,0.318612,13.000000,0.318137,"b'McCrie, Mr. James Matthew'",0.319484,0.410526,0.191987,0.329582,0.318137
1,37.0,0.022147,98,0.288326,0.318612,7.925000,0.313843,"b'Gustafsson, Mr. Anders Vilhelm'",0.319484,0.235412,0.191987,0.409086,0.318137
2,28.0,0.590473,-1,0.288326,0.318612,13.000000,0.448246,"b'Reynaldo, Ms. Encarnacion'",0.319484,0.410526,0.694006,0.329582,0.448246
3,18.0,0.022147,-1,0.288326,0.318612,73.500000,0.318137,"b'Davies, Mr. Charles Henry'",0.319484,0.410526,0.191987,0.329582,0.287396
4,-1.0,0.022147,-1,0.288326,0.523316,7.895800,0.253695,"b'Gheorgheff, Mr. Stanio'",0.319484,0.235412,0.191987,0.329582,0.318137
...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,21.0,0.576182,-1,0.288326,0.318612,10.500000,0.448246,"b'Rugg, Miss. Emily'",0.319484,0.410526,0.694006,0.329582,0.448246
912,36.0,0.022147,-1,0.288326,0.318612,7.895800,0.253695,"b'Turcin, Mr. Stjepan'",0.319484,0.235412,0.191987,0.329582,0.318137
913,-1.0,0.022147,-1,0.288326,0.318612,7.550000,0.253695,"b'MacKay, Mr. George William'",0.319484,0.235412,0.191987,0.329582,0.318137
914,39.0,0.524125,-1,0.288326,0.318612,211.337494,0.253695,"b'Kreuchen, Miss. Emilie'",0.319484,0.611354,0.694006,0.329582,0.455694


In [163]:
features = num_features + cat_features

In [164]:
ds_train[features]

,age,fare,boat,cabin,embarked,home.dest,parch,pclass,sex,sibsp,ticket
0,30.0,13.000000,0.022147,0.288326,0.318612,0.318137,0.319484,0.410526,0.191987,0.329582,0.318137
1,37.0,7.925000,0.022147,0.288326,0.318612,0.313843,0.319484,0.235412,0.191987,0.409086,0.318137
2,28.0,13.000000,0.590473,0.288326,0.318612,0.448246,0.319484,0.410526,0.694006,0.329582,0.448246
3,18.0,73.500000,0.022147,0.288326,0.318612,0.318137,0.319484,0.410526,0.191987,0.329582,0.287396
4,-1.0,7.895800,0.022147,0.288326,0.523316,0.253695,0.319484,0.235412,0.191987,0.329582,0.318137
...,...,...,...,...,...,...,...,...,...,...,...
911,21.0,10.500000,0.576182,0.288326,0.318612,0.448246,0.319484,0.410526,0.694006,0.329582,0.448246
912,36.0,7.895800,0.022147,0.288326,0.318612,0.253695,0.319484,0.235412,0.191987,0.329582,0.318137
913,-1.0,7.550000,0.022147,0.288326,0.318612,0.253695,0.319484,0.235412,0.191987,0.329582,0.318137
914,39.0,211.337494,0.524125,0.288326,0.318612,0.253695,0.319484,0.611354,0.694006,0.329582,0.455694


Произведем стандартизацию для всех предикторов для тренировочного набора данных и применим полученное преобразование для валидационной и тестовой выборок.

In [165]:
from sklearn.preprocessing import StandardScaler


ss = StandardScaler()
ds_train[features] = ss.fit_transform(ds_train[features])
ds_val[features] = ss.transform(ds_val[features])
ds_test[features] = ss.transform(ds_test[features])

Опишем метрики качества модели классификации

In [166]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# Построение Нейросети

In [167]:
input_shape = (len(features), )     # Наши данные - одномерный массив с размерностью 4
# batch_size = 10         # Размер батча на текущий момент можно выбрать любым, мы рассмотрим его оптимальный выбор далее в рамках курса
output_shape = 1  # Количество классов определено данными — их 3

## Model

In [168]:
model = keras.Sequential()
model.add(layers.InputLayer(input_shape = input_shape))

model.add(layers.Dense(32, activation = 'relu'))
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dense(128, activation = 'relu'))
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dense(32, activation = 'relu'))

model.add(layers.Dense(output_shape, activation = activations.sigmoid))

Воспользуемся модулем остановки обучения в случае, если показатель качества модели перестает существенно изменяться. А также модулем, который позволит нам сохранить в файл "лучшую" версию модели.

In [169]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
es_callback = EarlyStopping(patience=3)
filename = 'model.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_f1_m', verbose=1, save_best_only=True, mode='max')

Компилируем модель и производим обучение

In [170]:
model.compile(
    optimizer = optimizers.Adam(learning_rate = 0.003),
    loss = losses.BinaryCrossentropy(),
    metrics = [f1_m, precision_m, recall_m]
)

In [171]:
history = model.fit(ds_train[features],
                    y_train, 
                    epochs=30, batch_size=64, callbacks=[checkpoint, es_callback],
                    validation_data = (ds_val[features], y_val))

Epoch 1/30
14/15 [===========================>..] - ETA: 0s - loss: 0.3671 - f1_m: 0.9092 - precision_m: 0.8766 - recall_m: 0.9713 
Epoch 1: val_f1_m improved from -inf to 0.82457, saving model to model.h5
15/15 [==============================] - 2s 33ms/step - loss: 0.3622 - f1_m: 0.9124 - precision_m: 0.8849 - recall_m: 0.9676 - val_loss: 0.3648 - val_f1_m: 0.8246 - val_precision_m: 0.9472 - val_recall_m: 0.7596
Epoch 2/30
 1/15 [=>............................] - ETA: 0s - loss: 0.0538 - f1_m: 0.9412 - precision_m: 0.8889 - recall_m: 1.0000
Epoch 2: val_f1_m improved from 0.82457 to 0.90121, saving model to model.h5
15/15 [==============================] - 0s 9ms/step - loss: 0.0223 - f1_m: 0.9889 - precision_m: 0.9859 - recall_m: 0.9927 - val_loss: 0.6995 - val_f1_m: 0.9012 - val_precision_m: 0.9833 - val_recall_m: 0.8386
Epoch 3/30
14/15 [===========================>..] - ETA: 0s - loss: 0.0430 - f1_m: 0.9859 - precision_m: 0.9810 - recall_m: 0.9921
Epoch 3: val_f1_m did not improv

Выбираем "лучшую" модель, веса которой мы сохранили в файле

In [172]:
# load 
from keras.models import load_model
model = load_model('model.h5', custom_objects={'f1_m': f1_m, 'precision_m': precision_m, 'recall_m': recall_m})

In [173]:
from sklearn.metrics import f1_score

In [174]:
predict = model.predict(ds_test[features])

f1_score(y_test, predict>=0.5)

5/5 [==============================] - 0s 4ms/step


0.9838709677419354

# Предсказания

In [175]:
ds_prediction = tfds.as_dataframe(ds_prediction_tf)
ds_prediction = ds_prediction.drop(['survived'], axis=1)

In [176]:
ds_prediction = te.transform(ds_prediction)
ds_prediction[features] = ss.transform(ds_prediction[features])

In [177]:
ds_prediction[features]

,age,fare,boat,cabin,embarked,home.dest,parch,pclass,sex,sibsp,ticket
0,-1.267951,-0.093382,1.621461,-0.465767,-0.555806,1.196969,0.900583,0.285015,-0.727472,-0.573819,1.324843
1,-1.369134,-0.198626,1.164168,-0.465767,-0.238204,-0.836071,-0.505905,-0.828906,1.374624,0.468748,1.324843
2,-0.982794,1.837610,1.314345,2.429958,1.921751,1.196969,0.900583,1.562503,-0.727472,-0.573819,1.324843
3,0.286606,-0.450601,-0.736606,-0.465767,-0.555806,-0.836071,-0.505905,-0.828906,-0.727472,-0.573819,-0.003194
4,-1.369134,-0.624177,-0.736606,0.935642,-0.555806,-0.836071,-0.505905,1.562503,-0.727472,-0.573819,1.324843
...,...,...,...,...,...,...,...,...,...,...,...
126,0.452180,-0.482326,-0.736606,-0.465767,-0.238204,-0.255737,-0.505905,-0.828906,-0.727472,-0.573819,-0.003194
127,-0.265307,-0.481869,-0.736606,-0.465767,-0.555806,0.334589,-0.505905,-0.828906,-0.727472,-0.573819,-0.003194
128,-1.369134,-0.475234,-0.736606,-0.465767,-0.238204,-0.836071,-0.505905,-0.828906,1.374624,-0.573819,-0.003194
129,0.396989,-0.248502,0.652523,-0.465767,-0.555806,0.533636,2.367106,-0.828906,1.374624,1.689569,0.303331


In [180]:
import numpy as np

predict_platform = model.predict(ds_prediction[features])
predictions = np.array([1 if predict_platform[_] >= 0.5 else 0 for _ in range(len(predict_platform))])

5/5 [==============================] - 0s 2ms/step


In [185]:
predictions

array([1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0])